In [1]:
import re

prevnote = 0
prevref = 0
class py_solution:
    def roman_to_int(self, s):
        rom_val = {'I': 1, 'V': 5, 'X': 10, 'L': 50, 'C': 100, 'D': 500, 'M': 1000}
        int_val = 0
        for i in range(len(s)):
            if i > 0 and rom_val[s[i]] > rom_val[s[i - 1]]:
                int_val += rom_val[s[i]] - 2 * rom_val[s[i - 1]]
            else:
                int_val += rom_val[s[i]]
        return int_val
    
    
class py_solutionb:
    def int_to_Roman(self, num):
        val = [
            1000, 900, 500, 400,
            100, 90, 50, 40,
            10, 9, 5, 4,
            1
            ]
        syb = [
            "M", "CM", "D", "CD",
            "C", "XC", "L", "XL",
            "X", "IX", "V", "IV",
            "I"
            ]
        roman_num = ''
        i = 0
        while  num > 0:
            for _ in range(num // val[i]):
                roman_num += syb[i]
                num -= val[i]
            i += 1
        return roman_num


#print(py_solution().int_to_Roman(1))
    
def cleanline(s):
    s = re.sub('(\\b0\. A\.|<foreign xml:lang="grc">θ</foreign>\. A\.)','Θ. Δ.',s)
    s = re.sub('<lb/>','',s)
    s = re.sub('\s+$','',s)
    s = re.sub('^\s+','',s)
    s = re.sub('æ','ae',s)
    s = re.sub('Æ','Ae',s)
    s = re.sub(' 0 ',' O ',s)
    s = re.sub("''",'"',s)
    #if(re.search('\xad',s)):
    #    s = re.sub('\xad','-',s)
    #    print(s)
    s = re.sub('(<note type="footnote")>\s*([0-9])','\g<1> n="\g<2>">',s)
    s = re.sub('<note type="footnote">(\s*[A-Z]\s*|\s*VOL\.\s*[IV]+\.\s*|\s*[A-Z]\s*[0-9]+\s*)</note>','',s)
    return(s)

def additem(curitem,curdict):
    if(curitem in curdict):
        curdict[curitem] = curdict[curitem] + 1
        
    else:
        curdict[curitem] =  1

notelist = {}
reflist = {}

def countnotes(s,curpage):
    global notelist
    global reflist
    global prevnote
    global prevref
    
    saves = s
    
    while(re.search('<ref n="([0-9]+)"',s)):
        m = re.search('<ref n="([0-9]+)"',s)
        rnum = int(m[1])
        if(rnum != prevref + 1):
            print('refjump',curpage,prevref,rnum)
        prevref =rnum
        curref = curpage + '-' + m[1]
        additem(curref,reflist)
        s = re.sub('<ref n="([0-9]+)"','',s,1)
        
    s = saves
    
    while(re.search('<note type="footnote" n="([0-9]+)"',s)):
        m = re.search('<note type="footnote" n="([0-9]+)"',s)
        rnum = int(m[1])
        if(rnum != prevnote + 1):
            print('notejump',curpage,prevnote,rnum)
        prevnote =rnum
        curref = curpage + '-' + m[1]
        #print('sawnote',curref)
        additem(curref,notelist)
        s = re.sub('<note type="footnote" n="([0-9]+)"','',s,1)

def getspeaks(l):
    slist = []
    while(re.search('<label>([^<]+)',l)):
        m = re.search('<label>([^<]+)',l)
        slist.append(m[1])
        l = re.sub('<label>([^<]+)','',l,1)
    return(slist)
goodwords = {}


def dobohn(fname):
    i = 0
    global goodwords
    global prevref
    global prevnote
    speaklist = {}
    curwork = ''
    curpage = ''
    curchapter = 0
    prevchapter = 0
    prevspeak = ''
    f = open(fname)
    outfname = re.sub('.xml','-2.xml',fname)
    outf = open(outfname,'w')
    
    curspeaks = []
    
    for l in f:
        l = re.sub('\s+$','',l)
        #print(l)
        if(not l):
            continue
        l = re.sub('(<pb[^>]+>)','\n\g<1>\n',l)
        if(re.search('^<p>',l)):
            print('\n'+ l,file=outf,end=' ')
            
        elif(l[0] == '<' ):
            print('\n'+ l,file=outf)
        else:
            print(l,end=' ',file=outf)
        
    f.close()
    outf.close()
    
    f = open(outfname)
    outfname2 = re.sub('-2','-3',outfname)
    outf = open(outfname2,'w')
    printsect = 0
    inindex = 0
    innote = 0
    firsthalf = ''
    inbody = 0
    for l in f:
        l = cleanline(l)
        if(re.search('subtype="index"',l)):
            inindex = 1
            curspeaks = []
            prevspeak = ''
        l = re.sub('(https://epidoc.stoa.org/schema/latest/tei-epidoc.rng|http://www.stoa.org/epidoc/schema/latest/tei-epidoc.rng)','https://epidoc.stoa.org/schema/latest/tei-epidoc.rng',l)
        l = re.sub('\s+$','',l)
        l = re.sub('([a-z])\xad\s*([a-z])','\g<1>\g<2>',l)
        if(re.search('<body',l)):
            inbody = 1
        m = re.search('<pb n="([0-9]+)"',l)
        if(m):
            curpage = m[1]
        if(re.search('^<p>',l) and printsect):
            print('<div type="textpart" subtype="section" n="1">',file=outf)
            printsect = 0
            
        m = re.search('\s+([^ ]*[a-z])\xad',l)
        if(m):
            i = i + 1
            #print(i,m[1],l)
            firsthalf = m[1]
            l = re.sub('\s+([^ ]*[a-z])\xad',' ',l,1)
        
        m = re.search('n="(tlg[^"]+)"',l)
        if(m):
            curwork = m[1]
            curchapter = 0
            prevchapter = 0
            speaklist = {}
            print(curwork)
            #continue
            
        if(not l):
            continue
            
        if(re.search('<pb',l)):
            innote = 0
            prevref = 0
            prevnote = 0
            
        if(re.search('type="footnote"',l)):
            innote = 1
        
        if(re.search('tlg0',l) or re.search('subtype="book"',l)):
            curspeaks = []
            prevspeak = ''
            speaklist = {}
        
        if(re.search('<label>',l)):
            curspeaks = getspeaks(l)
            #print('curspeaks',curwork,curpage,curspeaks)
        if(l[0].islower() and firsthalf):
            #print(firsthalf+l)
            firsthalf = ''
        
        l = re.sub('\s*(<note)','\n\g<1>',l)
        
        if(re.search('subtype="chapter">',l)):
            prevchapter = 0
            curchapter = 0
        # for lucian
        m = re.search('(<ab><title type="sub">)([IVLXL]+)\.\s+',l)
        if(m):
            curromanchapter = m[2]
            #print('curr',curromanchapter,l)
            curchapter = py_solution().roman_to_int(curromanchapter)
            if( not curchapter == prevchapter + 1):
                print('chapjump',curwork,prevchapter,curchapter,l)
           # else:
           #     print(curchapter)
            if(curchapter!= 1):
                printsect = 1
                print('</div></div>\n\n<div type="textpart" subtype="book" n="'+str(curchapter)+'">',file=outf)
            #print('<div type="textpart" subtype="section" n="1">',file=outf)
            prevchapter = curchapter
            prevspeak = ''
            speaklist = {}

            
        m = re.search('<p>([IVXL]+)\.',l)
        if(m):
            curromanchapter = m[1]
            curchapter = py_solution().roman_to_int(curromanchapter)
            l = re.sub('<p>([IVXL]+)\.','<p><milestone unit="chapter" n="'+str(curchapter)+'"/>',l)
            if( not curchapter == prevchapter + 1):
                print('chapjump',curwork,prevchapter,curchapter,l)
           # else:
           #     print(curchapter)
            prevchapter = curchapter
            
        if(curspeaks):
            #m = re.search('(<p>)([A-Z][a-z]+|[A-Z][a-z]+\s+[A-Z][a-z+])\.\s+(\([^\)]+\))',l)
           # if(m):
           #     print(curpage,'speak',m[2],'stage',m[3])
            l = re.sub('(<p>)([A-Z][a-z]+|[A-Z][a-z]+\s+[A-Z][a-z+])\s+(\([^\)]+\))\.','\g<1>\g<2>. <stage>\g<3></stage>',l)
            m = re.search('<p>([A-Z][a-z]+|[A-Z][a-z]+\s+[A-Z][a-z]+)\.',l)
            if(m):
                curspeak = m[1]
                if(not curspeak in curspeaks):
                    print(curpage,curspeak,'not in',curspeaks)
                l = re.sub('<p>([A-Z][a-z]+|[A-Z][a-z]+\s+[A-Z][a-z]+)\.','<p><label>\g<1></label>',l)
                if(prevspeak and prevspeak == curspeak):
                    print(curpage,'repeatedspeak',curspeak,l)
                prevspeak = curspeak
        if(inbody and not innote and not inindex):
            l = re.sub('([,:\.\!a-z;"”\?])\s*([0-9])([ —<\(]|$)','\g<1><ref n="\g<2>"/>\g<3>',l)
            l = re.sub('\s+(\([^\)]+\))',' <stage>\g<1></stage>',l)
        if(not inindex):      
            countnotes(l,curpage)
        
        if(not re.search('<(head|title)',l) and re.search('<label>',l)):
            m = re.search('<label>([^<]+)',l)
            if(m):
                additem(m[1],speaklist)
            labelid = m[1]  + '-' + str(speaklist[m[1]])  
            l = re.sub('<label>','<label n="'+labelid+'">',l,1)
        print(l,file=outf)   
            
        

    f.close()
    outf.close()
    
    

#dobohn('/Users/gcrane/github/philoalex/yonge/philo_4_1855.xml')
#dobohn('/Users/gcrane/github/philoalex/yonge/philo_2_1854.xml')
#dobohn('/Users/gcrane/github/philoalex/yonge/philo_1_1854.xml')
#dobohn('/Users/gcrane/github/english_trans-dev/volumes/lucian_1888/lucian_1888.xml')
dobohn('lucian_1888.xml')


tlg0062.tlg068
tlg0062.tlg067
tlg0062.tlg066
tlg0062.tlg018
tlg0062.tlg017
tlg0062.tlg032
236 repeatedspeak Zeus <p><label>Zeus</label> Very just, Momus, and every one who is in favour  of it hold up his hand ; 2 or, rather, so let it take effect at  once: for I know that the dissentients will be in the majority.  The Assembly is now dismissed. But, whenever Hermes  shall make proclamation, come each of you with clear  testimonials, and plain proofs of your titles—the father's  and mother's names, and whence, and how, he or she became  a divinity, his tribe, and wardsmen.3 As, in the case of  whoever shall not exhibit these, whether a claimant has a  big temple upon the Earth, and whether he is regarded  by men as divine, the Commission will not trouble themselves.
tlg0062.tlg016
tlg0062.tlg035


In [129]:
for foo in reflist:
    if(not foo in notelist):
        print('missing note',foo)
        
for foo in notelist:
    if(not foo in reflist):
        print('missing ref',foo)
        
        

In [133]:
l = 'text (note) more text'

l = re.sub('\s+(\([^\)]+\))','<stage>\g<1></stage>',l)
l

'text<stage>(note)</stage> more text'

In [4]:
class py_solution:
    def roman_to_int(self, s):
        rom_val = {'I': 1, 'V': 5, 'X': 10, 'L': 50, 'C': 100, 'D': 500, 'M': 1000}
        int_val = 0
        for i in range(len(s)):
            if i > 0 and rom_val[s[i]] > rom_val[s[i - 1]]:
                int_val += rom_val[s[i]] - 2 * rom_val[s[i - 1]]
            else:
                int_val += rom_val[s[i]]
        return int_val

print(py_solution().roman_to_int('MMMCMLXXXVI'))
print(py_solution().roman_to_int('MMMM'))
print(py_solution().roman_to_int('C'))

3986
4000
100


In [12]:
import re

f = open('/Users/gcrane/scratch/philo/eng/sects21-ccel.txt')
curchapter = ''
outf = open('sects21-extra.txt','w')
for l in f:
    if(re.search('^Joseph\.',l)):
        continue
    l = re.sub('\s+$','',l)
    l = re.sub('^\s+','',l)
    m = re.search('^([IVXLC]+)\.',l)
    if(m):
        curchapter = py_solution().roman_to_int(m[1])
        l = re.sub('^([IVXLC]+)\.\s*','',l)
    while(re.search('\s*\(([0-9]+)\)\s+',l)):
        l = re.sub('\s*\(([0-9]+)\)\s+','\n' + str(curchapter) + ' '+ '\g<1> ',l,1)
    print(l,end=' ',file=outf)
print(file=outf)

outf.close()


In [22]:
import re
import os



bibabbs = {}
bibabbs['ge'] = 'Genesis\ttlg0527.tlg001'
bibabbs['ex'] = 'Exodus\t0tlg0527.tlg002'
bibabbs['le'] = 'Leviticus\ttlg0527.tlg004'
bibabbs['nu'] = 'Numbers\ttlg0527.tlg004'
bibabbs['de'] = 'Deuternomy\ttlg0527.tlg005'
bibabbs['jer'] = 'Jeremiah\ttlg0527.tlg036'


engbooks = {}

engbooks['1'] = ['01','De opificio mundi','On the Creation of the World','','','1']
engbooks['2'] = ['02','Legum allegoriarum libri i','On the allegories of the sacred laws i','1','3','1']
engbooks['3'] = ['02','Legum allegoriarum libri ii','On the allegories of the sacred laws ii','2','3','1']
engbooks['4'] = ['02','Legum allegoriarum libri iii','On the allegories of the sacred laws iii','3','3','1']
engbooks['5'] = ['03','De cherubim','On the Cherubim','','','1']
engbooks['6'] = ['04','De sacrificiis Abelis et Caini','On the sacrifice of Abel and Cain','','','1']
engbooks['7'] = ['05','Quod deterius potiori insidiari soleat','The worse attacks the better','','','1']
engbooks['8'] = ['06','De posteritate Caini','On the posterity and exile of Cain','','','1']
engbooks['9'] = ['07','De gigantibus','On the Giants','','','1']
engbooks['10'] = ['08','Quod deus sit immutabilis','On the unchangeableness of God i','','','1']
engbooks['11'] = ['09','De agricultura','On agriculture','','','1']
engbooks['12'] = ['10','De plantatione','On planting','','','1']
engbooks['13'] = ['11','De ebrietate','On drunkenness','','','1']
engbooks['14'] = ['12','De sobrietate','On sobriety','','','1']
engbooks['15'] = ['13','De confusione linguarum','On the confusion of languages','','','2']
engbooks['16'] = ['14','De migratione Abrahami','On the migration of Abraham','','','2']
engbooks['17'] = ['15','Quis rerum divinarum heres sit','Who is the heir?','','','2']
engbooks['18'] = ['16','De congressu eruditionis gratia','On the preliminary studies','','','2']
engbooks['19'] = ['17','De fuga et inventione','On flight and finding','','','2']
engbooks['20'] = ['18','De mutatione nominum','On the change of names','','','2']
engbooks['21'] = ['19','De somniis','On dreams','','','2']
engbooks['22'] = ['20','De Abrahamo','On the life of Abraham','','','2']
engbooks['23'] = ['21','De Josepho','On the life of Joseph','','','2']
engbooks['24'] = ['22','De vita Mosis','On the life of Moses','1','2','3']
engbooks['25'] = ['22','De vita Mosis','On the life ofMoses','2','2','3']
engbooks['26'] = ['23','De decalogo','On the Ten Commandments','2','2','3']
engbooks['27'] = ['24','De specialibus legibus','On the Special Laws','1','4','3']
engbooks['28'] = ['24','De specialibus legibus','On the Special Laws','2','4','3']
engbooks['29'] = ['24','De specialibus legibus','On the Special Laws','3','4','3']
engbooks['30'] = ['24','De specialibus legibus','On the Special Laws','4','4','3']
engbooks['31'] = ['25','De virtutibus','On the Virtues','','','3']
engbooks['32'] = ['26','De praemiis et poenis','On rewards and punishments','','','4']
engbooks['33'] = ['27','Quod omnis probus liber sit','Every good man ss free','','','4']
engbooks['34'] = ['28','De vita contemplativa','On the contemplative life','','','4']
engbooks['35'] = ['29','De aeternitate mundi','On the eternity of the world','','','4']
engbooks['36'] = ['30','In Flaccum','Against Flaccus','','','4']
engbooks['40'] = ['31','Legatio ad Gaium','On the embassy to Gaius','','','4']
engbooks['44'] = ['47','De mundo','On the world','','','4']

def makework2files(workn):
    filelist = []
    for foo in engbooks:
        #print(workn,engbooks[foo])
        curlist = engbooks[foo]
        if(int(curlist[0]) == workn):
            #print(foo,curlist[0],curlist[3])
            filelist.append([foo]+curlist)
    
    return(filelist)
            
work2files = {}
for i in range(1,48):
    work2files[i] = makework2files(i)
    #print('checking',i,work2files[i])
#work2files[47] = makework2files(47)

def dowork(htmln,outf):
    
    workinfo = engbooks[str(htmln)]
    tlgnum = workinfo[0]
    usetitle = workinfo[2]
    curbook = workinfo[3]
    totbooks = workinfo[4]
    yongevol = workinfo[5]
    curromanchap = ''
    #engoutfname = 'xml/tlg0018.tlg0' + tlgnum + '.yonge.xml'
    
    print('usetitle',usetitle)
    print('curbook',curbook,workinfo)
    f = open('book' + htmln+ '.html')
    bookoutf = open('book-work'+ str(i)+ '.html','w')
    
    intext = 0
    curnote = 0
    prevnoten = '0'
    noten = '0'
    curtitle = ''
    
    cursection = 0
    prevsection = 0
    closediv = '\n\n'
    curchapter = 0
    prevchapter = 0
    for l in f: #here
        if(re.search('earlychristianwritings',l)):
            continue
        #print(l)
        l = re.sub('\n','',l)
        l = re.sub('(<p>|</p>|<P>|</P>)','',l)
        l = re.sub('^\s+','',l)
        l = re.sub('�','—',l)
        
        

        
        if(re.search('<hr>',l)):
            intext = 1
            continue
        if(re.search('(<style|<a href=)',l)):
            intext = 0;
            continue
        if(not intext):
            continue
        if(l == ''):
            print('',file=bookoutf)
            continue
        if(not curtitle and re.search('[A-Z][A-Z][A-Z]',l)):
            curtitle = l
            print("curtitle",curtitle)
            continue
        if(l[0] == '<'):
            print('\n\n',file=bookoutf)
            
        m = re.search('^([IVXLC]+)\.\s*',l)
        if(m):
            curromanchap = m[1]
            #print(m[1],py_solution().roman_to_int(m[1]))
            curchapter = py_solution().roman_to_int(m[1])
            l = re.sub('^([IVXLC]+)\.\s*','<milestone unit="chapter" n="'+str(curchapter)+'"/>\n',l)
            if(not curchapter == prevchapter + 1):
                print('chapj',prevchapter,curchapter,l)
            #else:
            #    print("chapok",prevchapter,curchapter,l)
            prevchapter = curchapter
            
        l = re.sub('\([12]\.','(',l)     
        while(re.search('\s*(\([0-9]+\))',l)):

            m = re.search('\s*\(([0-9]+)\)',l)
            l = re.sub('\s*\(([0-9]+)\)',closediv + '\n\n<div type="textpart" subtype="section" n="\g<1>"><p>',l,1)
            closediv = '</p>\n</div>'
            cursection = int(m[1])
            if(not cursection == prevsection + 1):
                print('jmping from ',prevsection," to ",cursection)
            prevsection = cursection
        print(l,file=bookoutf,end=' ')

    bookoutf.close()
    f.close()
    
    f = open('book-work'+ str(i)+ '.html')
    #outf = open(engoutfname,'w')

    for l in f:
        #print(l)
        l = re.sub('\{\*+Yonge\'s title,\s+(.+)\}','<head>\g<1></head>',l)
        l = re.sub('^(PART [IV]+\.)','<head>\g<1></head>',l)

        l = re.sub('[ ]+',' ',l)
        l = re.sub('�e','ê',l)
        while(re.search('#([a-zA-Z]+)\s+([0-9:]+)',l)):
            m = re.search('#([a-zA-Z]+)\s+([0-9:]+)',l)
            curwork = m[1].lower()
            curcit = re.sub(':','.',m[2])
            if(not curwork in bibabbs):

                print('find',curwork)
                curworkname = 'tba'
                curworkid = 'tba'
            else:

                #print(curwork,bibabbs[curwork],curcit)
                    
                curworkname,curworkid = bibabbs[curwork].split('\t')
            newcit = '<bibl n="'+curworkid+':'+curcit+'">'+curworkname + ' '+curcit+'</bibl>'
                #print(newcit)
            l = re.sub('#([a-zA-Z]+)\s+([0-9:]+)',newcit,l,1)

        while(re.search('{[0-9]+}\s*{[^}]+}',l)):
                  
            m = re.search('{([0-9]+)}\s*{([^}]+)}',l)
            noten = m[1]
            if(not int(noten) == int(prevnoten) + 1):
                print('jumping from note '+ prevnoten + ' to ' + noten)
            prevnoten = noten
            curnote = m[2]
            l = re.sub('{([0-9]+)}\s*{([^}]+)}','<note n="\g<1>">\g<2></note>',l,1)
            #print(noten,curnote)
            #print(l)
        l = re.sub('([0-9]+):([0-9]+)','\g<1>.\g<2>',l)

        print(l,file=outf)

    f.close()
    

def dowork1(workn):
    curlist = work2files[workn]
    print(workn,curlist)
    #htmln = curlist[0][0]
    usetitle = curlist[0][3]
    yongevol = curlist[0][6]
    curbook = ''
    
    
    tlgnum = re.sub('^(.)$','0\g<1>',str(workn))
    engoutfname = 'xml/tlg0018.tlg0' + tlgnum + '.yonge.xml'
    outf = open(engoutfname,'w')
    headerfname = 'yonge' + yongevol + 'header.xml'
    headerf = open(headerfname)
    for l in headerf:
        l = re.sub('<title>TBA','<title>'+usetitle,l)

        print(l,end='',file=outf)
    print('<text><body>',file=outf)  
    print('<div type="edition">',file=outf)

    print('\ncreating',engoutfname)
    for foo in curlist:
        curhtml = foo[0]
        curbook = foo[4]
        if(curbook):
            print('<div type="textpart" subtype="book" n="'+str(curbook)+'">',file=outf)
        print(workn,curhtml,curbook)
        dowork(curhtml,outf)
        print('</p>',file=outf)
        if(curbook):
            print('</div></div>',file=outf)
    if(not curbook):
        print('</div>',file=outf)
    print('</div></body></text></TEI>',file=outf)
        
    outf.close()   
        #dowork(workn)
for i in range(22,23):
    
    dowork1(i)
    continue
    #fname = 'wget https://www.earlyjewishwritings.com/text/philo/book' + str(i)+ '.html'
    print(fname)
    #os.system(fname)

22 [['24', '22', 'De vita Mosis', 'On the life of Moses', '1', '2', '3'], ['25', '22', 'De vita Mosis', 'On the life ofMoses', '2', '2', '3']]

creating xml/tlg0018.tlg022.yonge.xml
22 24 1
usetitle On the life of Moses
curbook 1 ['22', 'De vita Mosis', 'On the life of Moses', '1', '2', '3']
curtitle ON THE LIFE OF MOSES, I* 
chapj 48 46 <milestone unit="chapter" n="46"/>
(269) But when he was on his road a very manifest sign met him in the way, showing him
chapj 46 50 <milestone unit="chapter" n="50"/>
(275) But when the king heard that he was now near at hand, he went forth with his guards to
22 25 2
usetitle On the life ofMoses
curbook 2 ['22', 'De vita Mosis', 'On the life ofMoses', '2', '2', '3']
curtitle ON THE LIFE OF MOSES, II* 


In [124]:

def dowork2(workn):
    return # files already built
    
    #workinfo = engbooks[str(workn)]
    #tlgnum = workinfo[0]
    #usetitle = workinfo[2]
    #curbook = workinfo[3]
    #totbooks = workinfo[4]
    #yongevol = workinfo[5]
    
    if(workn<10):
        tlgnum = '0' + str(workn)
    else:
        tlgnum = str(workn)
    curromanchap = ''
    xmlfname = 'xml/tlg0018.tlg0' + tlgnum + '.yonge.xml'

    f = open(xmlfname)
    print('opened',xmlfname)
    sectfname = 'sects'+tlgnum+'.txt'
    sectoutf = open(sectfname,'w')
    curchapter = ''
    cursection = ''
    for l in f:
        m = re.search('<milestone unit="chapter" n="([0-9]+)"/>',l)
        if(m):
            curchapter = m[1]
    
        m = re.search('<div type="textpart" subtype="section" n="([0-9]+)"><p>\s*(.+)',l)
            
        if(m):
            print(curchapter,m[1],m[2],file=sectoutf)
            
    
    
    f.close()
    sectoutf.close()

    
for i in range(1,32):
    dowork2(i)
dowork2(47)

opened xml/tlg0018.tlg001.yonge.xml
opened xml/tlg0018.tlg002.yonge.xml
opened xml/tlg0018.tlg003.yonge.xml
opened xml/tlg0018.tlg004.yonge.xml
opened xml/tlg0018.tlg005.yonge.xml
opened xml/tlg0018.tlg006.yonge.xml
opened xml/tlg0018.tlg007.yonge.xml
opened xml/tlg0018.tlg008.yonge.xml
opened xml/tlg0018.tlg009.yonge.xml
opened xml/tlg0018.tlg010.yonge.xml
opened xml/tlg0018.tlg011.yonge.xml
opened xml/tlg0018.tlg012.yonge.xml
opened xml/tlg0018.tlg013.yonge.xml
opened xml/tlg0018.tlg014.yonge.xml
opened xml/tlg0018.tlg015.yonge.xml
opened xml/tlg0018.tlg016.yonge.xml
opened xml/tlg0018.tlg017.yonge.xml
opened xml/tlg0018.tlg018.yonge.xml
opened xml/tlg0018.tlg019.yonge.xml
opened xml/tlg0018.tlg020.yonge.xml
opened xml/tlg0018.tlg021.yonge.xml
opened xml/tlg0018.tlg022.yonge.xml
opened xml/tlg0018.tlg023.yonge.xml
opened xml/tlg0018.tlg024.yonge.xml
opened xml/tlg0018.tlg025.yonge.xml
opened xml/tlg0018.tlg026.yonge.xml
opened xml/tlg0018.tlg027.yonge.xml
opened xml/tlg0018.tlg028.yo

In [88]:
work2files[47]

[]

In [18]:
goodwords

{'A': 28,
 'TREATISE': 14,
 'THE': 40,
 'ACCOUNT': 1,
 'OF': 24,
 'ON': 17,
 'WORLD,': 1,
 'CREATION': 1,
 'AS': 1,
 'GIVEN': 1,
 'BY': 2,
 'MOSES.': 1,
 'I.': 20,
 'other': 484,
 'lawgivers,': 1,
 'some': 317,
 'have': 874,
 'set': 32,
 'forth': 171,
 'what': 602,
 'they': 917,
 'con\xad': 83,
 'sidered': 2,
 'to': 6460,
 'be': 1417,
 'just': 92,
 'and': 7757,
 'reasonable,': 1,
 'in': 3675,
 'a': 3213,
 'naked': 17,
 'manner,': 59,
 'while': 135,
 'others,': 47,
 'investing': 1,
 'their': 801,
 'ideas': 22,
 'with': 1336,
 'an': 522,
 'of': 9091,
 'amplification,': 1,
 'sought': 8,
 'bewilder': 2,
 'the': 13610,
 'people,': 25,
 'by': 1616,
 'truth': 69,
 'under': 76,
 'heap': 2,
 'fabulous': 14,
 'inventions.': 2,
 'But': 639,
 'Moses,': 119,
 'rejecting': 8,
 'both': 193,
 'these': 384,
 'methods,': 1,
 'one': 916,
 'as': 1889,
 'inconsiderate,': 2,
 'careless,': 1,
 'unphilosophical,': 1,
 'full': 60,
 'trickery,': 1,
 'made': 247,
 'beginning': 65,
 'his': 1142,
 'laws': 35,
 'be

In [14]:
from bs4 import BeautifulSoup as BS
import re

totnotes = 0
f = open('lucian_1888.xml')

curpage1 = f.read() 

soup = BS(curpage1, 'xml')
    
notes = soup.find_all('note',{'type': ['footnote']})
p_tags = soup.find_all('div',{'subtype': ['section']})
print(len(p_tags),'notes',len(notes))
for foo in notes:
    totnotes= len(str(foo).split()) + totnotes
    print(totnotes,foo)


345 notes 643
61 <note n="1" type="footnote"><p>The fine allegory of the rivalry of Paideia and Techne (" Trade ")
may have been suggested to him, in part, by the charming idyll of  Moschus (in which Europe and Asia appear in a dream to the daughter  of Kadmus, and contend for her possession), as well as by the Choice of  Herakles of Prodikus and Xenophon. </p></note>
153 <note n="1" type="footnote"><p>The most interesting, and most meritorious, fact recorded in the life
of this highly interesting philosopher, who inclined in opinion chiefly to  the School of Plato, in practice to that of Antisthenes, is his fine remonstrance addressed to the people of Athens, who were contemplating the  introduction of the cruel " sports" of the Roman amphitheatres.  “ Refrain, Athenians," he protested, " from voting this, until you have  first pulled down your altar erected to Pity " (Dem. 51). See, too, his  very rational remark on sacrifices (Dem. 6). </p></note>
434 <note n="1" type="footnote"><p>

In [5]:
p_tags

[]